### Import Historical Data by IBK API

Import data from 01. 01. 2016 to 01. 11. 2025

In [ ]:
import datetime
from ib_insync import *
import nest_asyncio


#Connect to IBK
nest_asyncio.apply()
ib = IB()
ib.connect("127.0.0.1", 7497, clientId=1)

#Define Stock Contract
contract = Stock("QQQ", "SMART", "USD")

#Download historical data
dt = "" #For datetime, an empty string is interpreted as "now".
barsList = []

while True:
    bars = ib.reqHistoricalData(contract, endDateTime = dt, durationStr="20 D", barSizeSetting= "5 mins", whatToShow= "TRADES", useRTH= True, formatDate =1)

    if not bars:
        break

    barsList.append(bars)
    dt = bars[0].date  # This step is crucial for the loop to work:
                   # The first time, dt = "" → meaning "up to now". 
                   # After retrieving a block of 20 days, dt is set to bars[0].date, 
                   # i.e., the date of the first bar in the block just received.
                   # On the next iteration, we retrieve the 20 days preceding the first bar of the last received block.
    print("Retrieved data up to:", dt)


    if dt.date() < datetime.date(2015, 1, 1):
    # This sets the lower limit for the data we want to retrieve.
    # Note: IB downloads data in blocks, so if the first bar of the last block
    # is still within the range, it will also download the next block.
        break

#Save data to CSV
allBars = []
for bars in reversed(barsList):   # Loop through each block of bars. reversed() is used because the original order is from newest to oldest.
    for b in bars:                # Loop through each bar within the block
        allBars.append(b)         # Append the single bar to the final list
df = util.df(allBars)             # Convert the list of bars to a DataFrame
df.to_csv(f"{contract.symbol}_historical_data.csv", index = False)
print("Historical data saved")
ib.disconnect()

### SQL Data Cleaning

```sql
create table qqq_historical_data (date timestamp, open float, high float, low float, close float, volume int, average float, barcount int);

SET GLOBAL local_infile=1;

load data local infile 'C:/Users/nicho/Desktop/Trading Space/Progetto QQQ/QQQ_historical_data.csv' into table qqq_historical_data
fields terminated by ','
lines terminated by '\n'
ignore 1 lines
(date ,open ,high ,low ,close ,volume ,average ,barCount);

alter table qqq_historical_data add column date_ny datetime;

set sql_safe_updates = 0;

update qqq_historical_data set date_ny = convert_tz(date, 'UTC', 'America/New_York');

alter table qqq_historical_data drop column date, drop column volume, drop column average, drop column barcount;

create table qqq_historical_data1 select date_ny, open, high, low, close from qqq_historical_data where date(date_ny) >= '2015-01-01' and date(date_ny) <= '2025-12-01';

WITH
list_open as (select date(date_ny) as date, min(time(date_ny)) as open_time from qqq_historical_data1 group by date(date_ny)),
list_day_open as (select * from list_open where open_time != '09:30:00'),
list_close as (select date(date_ny) as date, max(time(date_ny)) as close_time from qqq_historical_data1 group by date(date_ny)),
list_day_close as (select * from list_close where close_time != '15:55:00'),
list_day as (select date, open_time, NULL AS close_time from list_day_open union all select date, NULL AS open_time, close_time from list_day_close)
delete from qqq_historical_data1 where date(date_ny) in (select date from list_day);

WITH
list_open as (select date(date_ny) as date, min(time(date_ny)) as open_time from qqq_historical_data1 group by date(date_ny)),
list_day_open as (select * from list_open where open_time != '09:30:00'),
list_close as (select date(date_ny) as date, max(time(date_ny)) as close_time from qqq_historical_data1 group by date(date_ny)),
list_day_close as (select * from list_close where close_time != '15:55:00'),
list_day as (select date, open_time, NULL AS close_time from list_day_open union all select date, NULL AS open_time, close_time from list_day_close)
select * from list_day;

```

### Environment Setup

In [ ]:
from tqdm import tqdm
import datetime as dt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import mysql.connector
password_sql = 'YOUR_PASSWORD'

qqq_connection_sql = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = f"{password_sql}",
    database = "qqq")

### Define Strategy Classes

In [ ]:
class trading_strategy:
    def __init__(self):
        self.initial_account = 0
        self.account = [self.initial_account]
        self.buy_pos = 0
        self.sell_pos = 0
        self.stop_loss = 0
        self.take_profit = 0
        self.trades_pnl = []
        self.fee = 0
        self.r = 0
        self.r_history = []
        self.opening_price = 0
        self.sell_price = 0
        self.sell_price_history = []
        self.buy_price = 0
        self.buy_price_history = []
        self.dates = []
        self.unit_trades = []
        self.unit_list = []
        

    def record_trade(self, date):
        self.units = 1
        self.unit_list.append(self.units)
        self.trades_pnl.append((self.sell_price - self.buy_price)*self.units - 2*self.units*self.fee)
        self.account.append(self.account[-1] + self.trades_pnl[-1])
        self.dates.append(date)
        self.buy_price_history.append(self.buy_price)
        self.sell_price_history.append(self.sell_price)
        self.r = ((self.sell_price - self.buy_price)/(abs(self.opening_price - self.stop_loss)))
        self.r_history.append(self.r)
        self.unit_trades.append(self.trades_pnl[-1]/self.units)
    
    def reset(self):
        self.buy_pos = 0
        self.sell_pos = 0
        self.opening_price = 0
        

In [ ]:
class trading_strategy_sim:
    def __init__(self):
        self.initial_account = 25000
        self.account = [self.initial_account]
        self.buy_pos = 0
        self.sell_pos = 0
        self.stop_loss = 0
        self.take_profit = 0
        self.trades_pnl = []
        self.fee = 0.0005
        self.r = 0
        self.r_history = []
        self.opening_price = 0
        self.sell_price = 0
        self.sell_price_history = []
        self.buy_price = 0
        self.buy_price_history = []
        self.dates = []
        self.unit_trades = []
        self.unit_list = []
        self.risk = 0

    def record_trade(self, date):
        self.units = int(min((self.account[-1]*0.01)/self.risk, (4*self.account[-1])/self.opening_price))
        self.unit_list.append(self.units)
        self.trades_pnl.append((self.sell_price - self.buy_price)*self.units - 2*self.units*self.fee)
        self.account.append(self.account[-1] + self.trades_pnl[-1])
        self.dates.append(date)
        self.buy_price_history.append(self.buy_price)
        self.sell_price_history.append(self.sell_price)
        self.r = ((self.sell_price - self.buy_price)/(abs(self.opening_price - self.stop_loss)))
        self.r_history.append(self.r)
        self.unit_trades.append(self.trades_pnl[-1]/self.units)
    
    def reset(self):
        self.buy_pos = 0
        self.sell_pos = 0
        self.opening_price = 0
        self.risk = 0

    def restart(self):
        self.__init__()

### Data for Strategies

In [ ]:
qqq_historical_data = pd.read_sql('Select * From qqq_historical_data1 where date(date_ny) >= "2016-01-01"', qqq_connection_sql)
print(qqq_historical_data.dtypes)

qqq_historical_data['date'] = qqq_historical_data['date_ny'].dt.date
qqq_historical_data['time'] = qqq_historical_data['date_ny'].dt.time

qqq_historical_data = qqq_historical_data.sort_values(by=["date", "time"]).reset_index(drop=True)


### Buy & Hold

In [ ]:
buy_hold_open = qqq_historical_data['open'].iloc[0]
buy_hold_close = qqq_historical_data.groupby(qqq_historical_data['date'])['close'].last()
buy_hold_account = buy_hold_close - buy_hold_open

### Buy & Hold Intraday

In [ ]:
buy_hold_intraday_open = qqq_historical_data.groupby(qqq_historical_data['date'])['open'].first()
buy_hold_intraday_close = qqq_historical_data.groupby(qqq_historical_data['date'])['close'].last()
buy_hold_intraday_pnl = buy_hold_intraday_close - buy_hold_intraday_open
buy_hold_intraday_account = buy_hold_intraday_pnl.cumsum()


### Base Strategy 

```sql
create table base_strategy
WITH
direction as (select date(date_ny) as date, case when (close - open) > 0 then 'Buy' when (close - open) < 0 then 'Sell' end as direction from qqq_historical_data1 where time(date_ny) = '09:30:00'),
entry_price as (select date(date_ny) as date, open as entry_price from qqq_historical_data1 where time(date_ny) = '09:35:00'),
close_price as (select date(date_ny) as date, close as close_price from qqq_historical_data1 where time(date_ny) = '15:55:00'),
base_strategy as (select direction.*, entry_price.entry_price, close_price.close_price from direction join entry_price on direction.date = entry_price.date join close_price on direction.date = close_price.date)
select * from base_strategy
```

In [ ]:
base_strategy_db = pd.read_sql('Select * from base_strategy', qqq_connection_sql)
print(base_strategy_db.dtypes)

basic_strategy_db = base_strategy_db.sort_values(by=["date"]).reset_index(drop=True)

In [ ]:
base_strategy_pnl = []
base_strategy_dates = []
for x in range(0,len(base_strategy_db)):
    if base_strategy_db['direction'].iloc[x] == 'Buy':
        base_strategy_realized = base_strategy_db['close_price'].iloc[x] - base_strategy_db['entry_price'].iloc[x]
        base_strategy_pnl.append(base_strategy_realized)
        base_strategy_dates.append(base_strategy_db['date'].iloc[x])
    elif base_strategy_db['direction'].iloc[x] == 'Sell':
        base_strategy_realized = base_strategy_db['entry_price'].iloc[x] - base_strategy_db['close_price'].iloc[x]
        base_strategy_pnl.append(base_strategy_realized)
        base_strategy_dates.append(base_strategy_db['date'].iloc[x])
base_strategy_account = np.cumsum(base_strategy_pnl)

### Basic Strategy VS Buy & Hold VS Buy & Hold Intraday

In [ ]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x = base_strategy_dates, y = base_strategy_account, mode = 'lines', name = 'Basic Strategy'))
fig1.add_trace(go.Scatter(x = buy_hold_account.index, y = buy_hold_account.values, mode = 'lines', name = 'Buy & Hold'))
fig1.add_trace(go.Scatter(x = buy_hold_intraday_account.index, y = buy_hold_intraday_account.values, mode = 'lines', name = 'Buy & Hold Intraday'))
fig1.update_layout(title = dict(text = "Basic Strategy VS Buy & Hold VS Buy & Hold Intraday", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

### Test 1

```sql
create temporary table target_list
WITH
sl_list as (select date(date_ny) as date, case when direction = 'Buy' then low when direction = 'Sell' then high end as stop_loss from (select qqq_historical_data1.*, base_strategy.* from qqq_historical_data1 join base_strategy on date(qqq_historical_data1.date_ny) = base_strategy.date) as full_table where time(date_ny) = '09:30:00'),
risk_list as (select *, round(entry_price - stop_loss,2) as risk from (select base_strategy.*, sl_list.stop_loss from base_strategy join sl_list on base_strategy.date = sl_list.date) as work1),
target_list as (select date, direction, entry_price, close_price, stop_loss, round((entry_price + risk*10),2) as target from risk_list)
select * from target_list;
```
---
```sql
create table test1_data select qqq_historical_data1.*, target_list.direction, target_list.stop_loss, target_list.target from qqq_historical_data1 join target_list on date(qqq_historical_data1.date_ny) = target_list.date
```

In [ ]:
test1_db = pd.read_sql('Select * from test1_data', qqq_connection_sql)
print(test1_db.dtypes)

test1_db['date'] = test1_db['date_ny'].dt.date
test1_db['time'] = test1_db['date_ny'].dt.time

test1_db = test1_db.sort_values(by=["date", "time"]).reset_index(drop=True)

In [ ]:
test1 = trading_strategy()

In [ ]:
for x in tqdm(range(0, len(test1_db))):

    # Entry Logic
    if test1_db['time'].iloc[x] == dt.time(9,35,0):
        if test1_db['direction'].iloc[x] == 'Buy':
            test1.buy_pos = 1
            test1.buy_price = test1_db['open'].iloc[x]
            test1.stop_loss = test1_db['stop_loss'].iloc[x]
            test1.take_profit = test1_db['target'].iloc[x]
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x], 'BUY at:', test1.buy_price,
                  'SL at:', test1.stop_loss, 'TP at:', test1.take_profit)

        if test1_db['direction'].iloc[x] == 'Sell':
            test1.sell_pos = 1
            test1.sell_price = test1_db['open'].iloc[x]
            test1.stop_loss = test1_db['stop_loss'].iloc[x]
            test1.take_profit = test1_db['target'].iloc[x]
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x], 'SELL at:', test1.sell_price,
                  'SL at:', test1.stop_loss, 'TP at:', test1.take_profit)

    # Long Position Management 
    if test1.buy_pos == 1 and test1.sell_pos == 0:
        if test1_db['low'].iloc[x] <= test1.stop_loss:
            test1.sell_pos = 1
            test1.sell_price = test1.stop_loss
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x],
                  'SELL STOP at:', test1.sell_price, '[close]')
        elif test1_db['high'].iloc[x] >= test1.take_profit:
            test1.sell_pos = 1
            test1.sell_price = test1.take_profit
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x],
                  'SELL TP at:', test1.sell_price, '[close]')

    # Short Position Management
    if test1.buy_pos == 0 and test1.sell_pos == 1:
        if test1_db['high'].iloc[x] >= test1.stop_loss:
            test1.buy_pos = 1
            test1.buy_price = test1.stop_loss
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x],
                  'BUY STOP at:', test1.buy_price, '[close]')
        elif test1_db['low'].iloc[x] <= test1.take_profit:
            test1.buy_pos = 1
            test1.buy_price = test1.take_profit
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x],
                  'BUY TP at:', test1.buy_price, '[close]')
            
    # End Day
    if test1_db['time'].iloc[x] == dt.time(15,55,0):

        if test1.buy_pos == 1 and test1.sell_pos == 0:
            test1.sell_pos = 1
            test1.sell_price = test1_db['close'].iloc[x]
            print('SELL at:', test1_db['close'].iloc[x], '[close]')

        if test1.buy_pos == 0 and test1.sell_pos == 1:
            test1.buy_pos = 1
            test1.buy_price = test1_db['close'].iloc[x]
            print('BUY at:', test1_db['close'].iloc[x], '[close]')

    #Trade Record
    if test1.buy_pos == 1 and test1.sell_pos == 1:
        test1.record_trade(test1_db['date'].iloc[x])
        test1.reset()



### Test 1 VS Base Strategy VS Buy Hold Intraday

In [ ]:
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x = test1.dates, y = test1.account, mode = 'lines', name = 'Test 1'))
fig2.add_trace(go.Scatter(x = base_strategy_dates, y = base_strategy_account, mode = 'lines', name = 'Base Strategy'))
fig2.add_trace(go.Scatter(x = buy_hold_intraday_account.index, y = buy_hold_intraday_account.values, mode = 'lines', name = 'Buy & Hold Intraday'))
fig2.update_layout(title = dict(text = "Test 1 VS Base Strategy VS Buy & Hold Intraday", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

### Test 1 Money Management

In [ ]:
test2 = trading_strategy_sim()

# test2 = Test 1 Money Management

In [ ]:
for x in tqdm(range(0, len(test1_db))):
    # Entry Logic
    if test1_db['time'].iloc[x] == dt.time(9,35,0):
        if test1_db['direction'].iloc[x] == 'Buy':
            test2.buy_pos = 1
            test2.buy_price = test1_db['open'].iloc[x]
            test2.opening_price = test2.buy_price
            test2.stop_loss = test1_db['stop_loss'].iloc[x]
            test2.take_profit = test1_db['target'].iloc[x]
            test2.risk = abs(test2.opening_price - test2.stop_loss)
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x], 'BUY at:', test2.buy_price,
                  'SL at:', test2.stop_loss, 'TP at:', test2.take_profit)

        if test1_db['direction'].iloc[x] == 'Sell':
            test2.sell_pos = 1
            test2.sell_price = test1_db['open'].iloc[x]
            test2.opening_price = test2.sell_price
            test2.stop_loss = test1_db['stop_loss'].iloc[x]
            test2.take_profit = test1_db['target'].iloc[x]
            test2.risk = abs(test2.opening_price - test2.stop_loss)
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x], 'SELL at:', test2.sell_price,
                  'SL at:', test2.stop_loss, 'TP at:', test2.take_profit)

    # Long Position Management 
    if test2.buy_pos == 1 and test2.sell_pos == 0:
        if test1_db['low'].iloc[x] <= test2.stop_loss:
            test2.sell_pos = 1
            test2.sell_price = test2.stop_loss
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x],
                  'SELL STOP at:', test2.sell_price, '[close]')
        elif test1_db['high'].iloc[x] >= test2.take_profit:
            test2.sell_pos = 1
            test2.sell_price = test2.take_profit
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x],
                  'SELL TP at:', test2.sell_price, '[close]')

    # Short Position Management
    if test2.buy_pos == 0 and test2.sell_pos == 1:
        if test1_db['high'].iloc[x] >= test2.stop_loss:
            test2.buy_pos = 1
            test2.buy_price = test2.stop_loss
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x],
                  'BUY STOP at:', test2.buy_price, '[close]')
        elif test1_db['low'].iloc[x] <= test2.take_profit:
            test2.buy_pos = 1
            test2.buy_price = test2.take_profit
            print(test1_db['date'].iloc[x], test1_db['time'].iloc[x],
                  'BUY TP at:', test2.buy_price, '[close]')

    # End Day
    if test1_db['time'].iloc[x] == dt.time(15,55,0):

        if test2.buy_pos == 1 and test2.sell_pos == 0:
            test2.sell_pos = 1
            test2.sell_price = test1_db['close'].iloc[x]
            print('SELL at:', test1_db['close'].iloc[x], '[close]')

        if test2.buy_pos == 0 and test2.sell_pos == 1:
            test2.buy_pos = 1
            test2.buy_price = test1_db['close'].iloc[x]
            print('BUY at:', test1_db['close'].iloc[x], '[close]')

    # Trade Record
    if test2.buy_pos == 1 and test2.sell_pos == 1:
        test2.record_trade(test1_db['date'].iloc[x])
        test2.reset()


### Buy & Hold Money Management

In [ ]:
buy_hold_open1 = qqq_historical_data['open'].iloc[0]
buy_hold_units1 = int(25000/buy_hold_open1)
buy_hold_close1 = qqq_historical_data.groupby(qqq_historical_data['date'])['close'].last()
buy_hold_account1 = 25000 + (buy_hold_close1 - buy_hold_open1)*buy_hold_units1

### Test 1 Money Management VS Buy &  Hold Money Management 

In [ ]:
fig4 = go.Figure()
fig4.add_trace(go.Scatter(x = test2.dates, y = test2.account, mode = 'lines', name = 'Test 1'))
fig4.add_trace(go.Scatter(x = buy_hold_account1.index, y = buy_hold_account1.values, mode = 'lines', name = 'Buy & Hold'))
fig4.update_layout(title = dict(text = "Test 1 MM VS Buy & Hold MM", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

 ### Traded Units Test 1 Money Management

In [ ]:
fig5 = go.Figure()
fig5.add_trace(go.Scatter(x = test2.dates, y = test2.unit_list, mode = 'lines', name = 'Test 1'))
fig5.update_layout(title = dict(text = "Traded Units Test 1 MM", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))

In [ ]:
print("Average Traded Units per Trade:", np.mean(test2.unit_list))

In [ ]:
print("Average Profit per Unit:", np.mean(test2.unit_trades)) 

### Test 1 MM + Slippage Simulation

In [ ]:
class trading_strategy_sim_slip:
    def __init__(self):
        # --- Account ---
        self.initial_account = 25000
        self.account = [self.initial_account]

        # --- Position state ---
        self.buy_pos = 0
        self.sell_pos = 0

        # --- Prices (theoretical) ---
        self.buy_price = 0
        self.sell_price = 0
        self.opening_price = 0
        self.stop_loss = 0
        self.take_profit = 0

        # --- Prices (real, with slippage) ---
        self.real_buy_price = 0
        self.real_sell_price = 0

        # --- Trade info ---
        self.units = 0
        self.risk = 0
        self.fee = 0.0005

        # --- Stats ---
        self.trades_pnl = []
        self.unit_list = []
        self.r_history = []
        self.dates = []
        self.unit_trade = []

    # ===============================
    # POSITION SIZING (NO SLIPPAGE)
    # ===============================
    def compute_units(self):
        self.units = int(
            min(
                (self.account[-1] * 0.01) / self.risk,
                (4 * self.account[-1]) / self.opening_price
            )
        )
        self.unit_list.append(self.units)

    # ======================================
    # WEIGHTED PRICE WITH STEP SLIPPAGE
    # ======================================
    def weighted_price(self, price, units, side, step=750, slip=0.01):
        remaining = units
        level = 0
        total_cost = 0

        while remaining > 0:
            qty = min(step, remaining)

            if side == 'Buy':
                exec_price = price + level * slip   # peggiora verso l'alto
            else:  # Sell
                exec_price = price - level * slip   # peggiora verso il basso

            total_cost += qty * exec_price
            remaining -= qty
            level += 1

        return total_cost / units


    # ===============================
    # ENTRY EXECUTION
    # ===============================
    def apply_entry(self, direction):
        self.compute_units()

        if direction == 'Buy':
            self.real_buy_price = self.weighted_price(
                self.buy_price, self.units, side='Buy'
            )

        elif direction == 'Sell':
            self.real_sell_price = self.weighted_price(
                self.sell_price, self.units, side='Sell'
            )


    # ===============================
    # STOP EXECUTION 
    # ===============================
    def apply_stop(self, direction):
        if direction == 'Buy':  
            self.real_sell_price = self.weighted_price(
                self.sell_price, self.units, side='Sell'
            )

        elif direction == 'Sell':  
            self.real_buy_price = self.weighted_price(
                self.buy_price, self.units, side='Buy'
            )

    # ===============================
    # RECORD TRADE
    # ===============================
    def record_trade(self, date):
        pnl = (
            (self.real_sell_price - self.real_buy_price) * self.units
            - 2 * self.units * self.fee
        )

        self.trades_pnl.append(pnl)
        self.account.append(self.account[-1] + pnl)

        r = (self.sell_price - self.buy_price) / abs(
            self.opening_price - self.stop_loss
        )

        self.r_history.append(r)
        self.dates.append(date)
        self.unit_trade.append(self.trades_pnl[-1]/self.units)

    # ===============================
    # RESET SINGLE TRADE
    # ===============================
    def reset(self):
        self.buy_pos = 0
        self.sell_pos = 0

        self.buy_price = 0
        self.sell_price = 0
        self.opening_price = 0
        self.stop_loss = 0
        self.take_profit = 0

        self.real_buy_price = 0
        self.real_sell_price = 0

        self.units = 0
        self.risk = 0

    def restart(self):
        self.__init__

In [ ]:
test3 = trading_strategy_sim_slip()

# test3 = Test 1 MM + Slippage Sim

In [ ]:
for x in tqdm(range(0, len(test1_db))):

    # ================= ENTRY =================
    if test1_db['time'].iloc[x] == dt.time(9,35,0):

        if test1_db['direction'].iloc[x] == 'Buy':
            test3.buy_pos = 1
            test3.buy_price = test1_db['open'].iloc[x]
            test3.opening_price = test3.buy_price
            test3.stop_loss = test1_db['stop_loss'].iloc[x]
            test3.take_profit = test1_db['target'].iloc[x]
            test3.risk = abs(test3.opening_price - test3.stop_loss)

            test3.apply_entry('Buy')

        if test1_db['direction'].iloc[x] == 'Sell':
            test3.sell_pos = 1
            test3.sell_price = test1_db['open'].iloc[x]
            test3.opening_price = test3.sell_price
            test3.stop_loss = test1_db['stop_loss'].iloc[x]
            test3.take_profit = test1_db['target'].iloc[x]
            test3.risk = abs(test3.opening_price - test3.stop_loss)

            test3.apply_entry('Sell')

    # ================= LONG =================
    if test3.buy_pos == 1 and test3.sell_pos == 0:

        if test1_db['low'].iloc[x] <= test3.stop_loss:
            test3.sell_pos = 1
            test3.sell_price = test3.stop_loss

            test3.apply_stop('Buy')

        elif test1_db['high'].iloc[x] >= test3.take_profit:
            test3.sell_pos = 1
            test3.sell_price = test3.take_profit
            test3.real_sell_price = test3.sell_price  # NO slippage

    # ================= SHORT =================
    if test3.buy_pos == 0 and test3.sell_pos == 1:

        if test1_db['high'].iloc[x] >= test3.stop_loss:
            test3.buy_pos = 1
            test3.buy_price = test3.stop_loss

            test3.apply_stop('Sell')

        elif test1_db['low'].iloc[x] <= test3.take_profit:
            test3.buy_pos = 1
            test3.buy_price = test3.take_profit
            test3.real_buy_price = test3.buy_price  # NO slippage

    # ================= END DAY =================
    if test1_db['time'].iloc[x] == dt.time(15,55,0):

        if test3.buy_pos == 1 and test3.sell_pos == 0:
            test3.sell_pos = 1
            test3.sell_price = test1_db['close'].iloc[x]
            test3.real_sell_price = test3.sell_price

        if test3.buy_pos == 0 and test3.sell_pos == 1:
            test3.buy_pos = 1
            test3.buy_price = test1_db['close'].iloc[x]
            test3.real_buy_price = test3.buy_price

    # ================= RECORD =================
    if test3.buy_pos == 1 and test3.sell_pos == 1:
        test3.record_trade(test1_db['date'].iloc[x])
        test3.reset()


### Test 1 MM VS Test 1 MM + Slippage Simulation VS Buy & Hold

In [ ]:
fig6 = go.Figure()
fig6.add_trace(go.Scatter(x = test2.dates, y = np.cumsum(test2.trades_pnl), mode = 'lines', name = 'Test 1'))
fig6.add_trace(go.Scatter(x = test3.dates, y = np.cumsum(test3.trades_pnl), mode = 'lines', name = 'Test 1 + Slippage'))
fig6.add_trace(go.Scatter(x = buy_hold_account1.index, y = buy_hold_account1.values - 25000, mode = 'lines', name = 'Buy & Hold'))
fig6.update_layout(title = dict(text = "Test 1 MM VS Test 1 MM + Slippage Sim. VS Buy & Hold", yanchor = "top", font= dict(size = 30)), 
                  margin=dict(l=50, r=50, t=70, b=50), 
                  plot_bgcolor = "rgba(64,64,64,0.8)",
                  paper_bgcolor = "rgba(64,64,64,0.8)",
                  font = dict(color = "white"),
                  yaxis=dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)", zerolinewidth = 1),
                  xaxis = dict(showgrid = True, gridcolor = "rgba(255,255,255,0.3)"))